In [3]:
import random
import datasets
import spacy
from collections import Counter
from sklearn import metrics
import spacy
from collections import Counter
import torch
from transformers import BertTokenizer, BertForMaskedLM
import spacy
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
nlp = spacy.load('en_core_web_sm')

model = BertForMaskedLM.from_pretrained('allenai/scibert_scivocab_uncased')
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
    return torch.exp(loss).item()

In [6]:
def load_pubmed():
    data = datasets.load_dataset('pubmed_qa', 'pqa_labeled', split='train')

    # combine question and long_answer and add label, and create a spaCy Doc for each text
    data = [(nlp(f'Question: {q} Answer: {a}'), 0) for q, a in zip(data['question'], data['long_answer'])]

    return data


In [9]:

# Initialize counters
pos_counts = Counter()
function_word_counts = Counter()
punctuation_counts = Counter()
uppercase_counts = Counter()

# Initialize variables for sentence length calculation
total_tokens = 0
total_sentences = 0

# Initialize list to store each text's perplexity
perplexities = []

data = load_pubmed()


Found cached dataset pubmed_qa (C:/Users/atana/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/dd4c39f031a958c7e782595fa4dd1b1330484e8bbadd4d9212e5046f27e68924)


In [ ]:
for doc, _ in data:
    text = doc.text
    perplexities.append(calculate_perplexity(text))
    for token in doc:
        pos_counts[token.pos_] += 1
        if token.text in {'a', 'in', 'of'}:
            function_word_counts[token.text] += 1
        if token.is_punct:
            punctuation_counts[token.text] += 1
        if token.text.isupper():
            uppercase_counts[token.text] += 1
    total_tokens += len(doc)
    total_sentences += len(list(doc.sents))


In [ ]:
# Calculate average sentence length
avg_sentence_length = total_tokens / total_sentences if total_sentences > 0 else 0

# Calculate average perplexity
avg_perplexity = sum(perplexities) / len(perplexities) if perplexities else 0


In [ ]:
fig, ax = plt.subplots()

# Generate a KDE plot with seaborn
sns.kdeplot(perplexities, ax=ax)

# Set labels and title
ax.set_xlabel('Perplexity')
ax.set_ylabel('Density')
ax.set_title('Distribution of Perplexities')

# Show the plot
plt.show()